<a href="https://colab.research.google.com/github/exterm/PoCS-PrivacyPolicies/blob/main/negation_filtered_policytext_PII.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
import pandas as pd
import re
import numpy as np
import array as ar

#--
import os

In [13]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Summary 

1.   We are going to want to feed in each year of the privacy policy text
2.   then we want to filter out clauses that only have negation words (save the csv here) 
3.   then filter the data frame so it only has PII lexicon terms (save the csv here too)



In [14]:
#we will want to feed in each year of priviacy policies
data_base_path = '//content/drive/MyDrive/RESEARCH/CODE AND DATA/Pirvacy_Policies_OT/PoCS/data'
cleaned_data_path = 'by-year'
data_path = os.path.join(data_base_path, cleaned_data_path)
#--
df = pd.read_csv(os.path.join(data_path, 'PrivacyPolicies-1997.csv'))

In [15]:
df.head()

,site_id,policy_url,year,policy_title,policy_text,length,simhash,domain,categories,rank
0,105228,ssa.gov/privacy.html,1997,Social Security Online Privacy Policy,| \n\nThe privacy of our customers has always ...,1216,8312335261154066328,ssa.gov,business;education,NaN
1,50905,hbo.com/cmp/HBOPrivacyNotice.html,1997,HBO PRIVACY NOTICE,HBO PRIVACY NOTICE\n\nPRIVACY PRACTICES AND PO...,1636,13416900284197449768,hbo.com,education;entertainment,NaN
2,22041,cinemax.com/cmp/CinemaxPrivacyNotice.html,1997,CINEMAX PRIVACY NOTICE,CINEMAX PRIVACY NOTICE PRIVACY PRACTICES AND P...,1667,10246366146546577448,cinemax.com,entertainment;streamingmedia,NaN
3,8512,army.mil/security.htm,1997,Security and Privacy Notice,\n| | \n\n 1. This site is provided as a p...,710,6216778325323013202,army.mil,education,NaN
4,11498,bakersfield.com/privacy,1997,The Bakersfield Californian: Privacy Policy,This is the web site of The Bakersfield Califo...,1839,5766445921680763988,bakersfield.com,business;entertainment;newsandmedia,NaN


#negation 

In [16]:

df['list_of_clauses'] = df['policy_text'].astype('object')
df['nb_of_clauses'] = df['length']
df['list_of_clauses'].astype(str).str.lower()
df['nb_of_clauses'].astype(str).str.lower()

0     1216
1     1636
2     1667
3      710
4     1839
5     1596
6    13753
7     3442
8     3442
Name: nb_of_clauses, dtype: object

In [17]:
df_clauses = df[['nb_of_clauses', 'list_of_clauses']].copy()
df_clauses

,nb_of_clauses,list_of_clauses
0,1216,| \n\nThe privacy of our customers has always ...
1,1636,HBO PRIVACY NOTICE\n\nPRIVACY PRACTICES AND PO...
2,1667,CINEMAX PRIVACY NOTICE PRIVACY PRACTICES AND P...
3,710,\n| | \n\n 1. This site is provided as a p...
4,1839,This is the web site of The Bakersfield Califo...
5,1596,1\. This World Wide Web (WWW) site is provided...
6,13753,"August 8, 1997\n\n| | \n\n \nPart 1 in a 3-p..."
7,3442,## \n\n## The McGraw-Hill Companies Customer P...
8,3442,## \n\n## The McGraw-Hill Companies Customer P...


In [18]:
#we loop all rows and find all punctuation then split them each into a new column 
#every entry in the new column now becomes a list of clauses

for index, row in df_clauses.iterrows():
    policy_texts_split = re.split(r"[!\"#\$%&\'\(\)\*\+,-\./:;<=>\?@\[\\\]\^_`{\|}~]", df_clauses.at[index,'list_of_clauses'])
    policy_texts_split = list(filter(None, policy_texts_split))
    df_clauses.at[index,'list_of_clauses'] = policy_texts_split
    df_clauses.at[index,'nb_of_clauses'] = len(policy_texts_split)


In [19]:
df_clauses

,nb_of_clauses,list_of_clauses
0,32,[ \n\nThe privacy of our customers has always ...
1,31,[HBO PRIVACY NOTICE\n\nPRIVACY PRACTICES AND P...
2,31,[CINEMAX PRIVACY NOTICE PRIVACY PRACTICES AND ...
3,24,"[ \n, , \n\n 1, This site is provided as..."
4,51,[This is the web site of The Bakersfield Calif...
5,33,"[1, This World Wide Web , WWW, site is provi..."
6,489,"[August 8, 1997\n\n, , \n\n \nPart 1 in a..."
7,93,"[ \n\n, The McGraw, Hill Companies Customer P..."
8,93,"[ \n\n, The McGraw, Hill Companies Customer P..."


In [20]:
#negation words from Juniper and Peter 
negation_words = ["don't","never", "nothing", "nowhere", "noone", "none", "not",
                  "hasn't","hadn't","can't","couldn't","shouldn't","won't",
                  "wouldn't","don't","doesn't","didn't","isn't","aren't","ain't", "in*",  "un*", "dis*", "mal*"]


#“don't|never| nothing|nowhere|noone|none|not|hasn't|hadn't|can't|couldn't|shouldn't|won't|wouldn't|don't|doesn't|didn't|isn't|aren't|ain't|in*|un*|dis*|mal*”

In [21]:
#iterate through all the rows 
#look at list items that contain negation words
#if the list item contains a negation word then remove the list item (sentence)
#return a column with just the list of sentences that do not have negation words in them 

def remove_negations(text):
    # Step 1: split the text into sentences
    sent = re.split(r'[.!]', text)
    #list to store sentences without negataions
    w_neg = []
    #step 2: check for each sentence in row
    for i in sent:
      neg = False
      #step 3: check for each negation word in the list of negation words
      for j in negation_words:
        if j in i: #if there is a negataion word in sentence
          neg = True #make it true
          break; #break the loop and go for another sentence
      if not neg: #but if it's still false meaning that no negation word was found in a sentence
        w_neg.append(i) #add that sentence to the list

    return w_neg #now join back the sentences

df['negation_Extracted'] = df['policy_text'].apply(remove_negations) #apply at each row



In [30]:
df['negation_Extracted']

0    [| \n\nThe privacy of our customers has always...
1    [HBO PRIVACY NOTICE\n\nPRIVACY PRACTICES AND P...
2    [CINEMAX PRIVACY NOTICE PRIVACY PRACTICES AND ...
3    [  \n|  | \n\n  1,  This site is provided as a...
4    [This is the web site of The Bakersfield Calif...
5    [1\,  This World Wide Web (WWW) site is provid...
6    [ While it's true that the idea of information...
7    [## \n\n## The McGraw-Hill Companies Customer ...
8    [## \n\n## The McGraw-Hill Companies Customer ...
Name: negation_Extracted, dtype: object

In [25]:
#save it as a csv
#make sure to change the file name for the correct year 

df.to_csv('/content/drive/MyDrive/RESEARCH/CODE AND DATA/Pirvacy_Policies_OT/PoCS/data/negation_filtered/1997_negation_extracted.csv') 

#filter PII

In [26]:
#these are all of the terms in our PII lexicon put into a list 
PII = ['Identifiers',
    'alias',
    'online identifier',
    'Internet Protocol (IP) address',
    'account name',
    'social security number',
    'passport number',
    'Customer records information',
    'IDENTIFICATION NUMBER',
    'signature',
    'electronic mail address',
    'address',
    'telephone number',
    'PROTECTED HEALTH INFORMATION',
    'state identification card number',
    'education',
    'employment history',
    'bank account number',
    'face',
    'financial information',
    'Records of personal property',
    'products purchased',
    'health condition',
    'consuming histories',
    'Services purchased',
    'eye color',
    'retina scans',
    'network activity',
    'Internet  activity',
    'search history',
    'Geolocation',
    'visual',
    'thermal',
    'olfactory',
    'Professional',
    'Medical Condition',
    'Characteristics',
    'Aggregated data',
    'Predispositions',
    'Behavior',
    'specific location',
    'facial recognition',
    'physiological',
    'behavioral',
    'Audio',
    'DNA',
    'iris',
    'retina',
    'hand',
    'palm',
    'vein patterns',
    'voice recordings',
    'minutiae template',
    'HEALTH STATUS',
    'keystroke patterns ',
    'keystroke rhythms',
    'gait rhythms',
    'sleep',
    'health',
    'exercise',
    'Cross-context behavioral advertising',
    'targeted advertising',
    'Dark pattern',
    'Personal information',
    'RACIAL',
    'real name',
    'Account number',
    'postal address',
    'financial account number',
    'Internet Protocol address',
    'Gender identity',
    'email address',
    'security question',
    'Color',
    'Religion',
    'Sex',
    'Sexual orientation',
    'Marital status',
    'National origin',
    'Ancestry',
    'Genetic information',
    'Retaliation for reporting patient abuse in tax-supported institutions',
    'Age',
    'religious dress',
    'pregnancy',
    'Gender',
    'childbirth',
    'breastfeeding',
    'mental characteristics',
    'physical characteristics',
    'HIV/AIDS',
    'cancer',
    'genetic characteristics',
    'Geolocation data',
    'record of cancer',
    'history of cancer',
    'gender expression',
    'abilities',
    'MENTAL CONDITION',
    'PREDICT',
    'biological',
    'purchasing tendencies',
    'Aggregate consumer information',
    'first name',
    'voice',
    'electronic network activity',
    'biological characteristic',
    'interaction with an advertisement',
    'Browsing history',
    'employment',
    'Race',
    'health records',
    'citizenship',
    'Military or veteran status',
    'medical identification number',
    'access code',
    'preferences',
    'protected classifications',
    'psychological trends',
    'Commercial information',
    'medical information',
    'attitudes',
    'intelligence',
    'Name',
    'driver\'s license',
    'Sensitive personal information',
    'Precise geolocation',
    'locate',
    'geographic area',
    'radius',
    'Sensitive data',
    'Profiling',
    'consumer\'s social security',
    'driver\'s license',
    'state Identification card',
    'account log-In',
    'credit card',
    'health insurance information',
    'password',
    'credentials allowing access to an account',
    'combination',
    'racial origin',
    'ethnic origin',
    'philosophical beliefs',
    'union membership',
    'text messages',
    'genetic data',
    'biometric data',
    'Personally identifiable information',
    'security code',
    'fingerprint',
    'device identifier',
    'IP address',
    'cookies',
    'beacons',
    'pixel tags',
    'customer number',
    'unique pseudonym',
    'telephone numbers',
    'persistent identifier',
    'probabilistic identifier',
    'family',
    'child',
    'identifier template',
    'DE-IDENTIFIED DATA',
    'HEALTH-CARE INFORMATION',
    'HEALTH-CARE PROVIDER',
    'MEDICINE',
    'PHARMACY',
    'CHIROPRACTIC',
    'NURSING',
    'PHYSICAL THERAPY',
    'PODIATRY',
    'DENTISTRY',
    'OPTOMETRY',
    'OCCUPATIONAL THERAPY',
    'HEALING ARTS',
    'IDENTIFIED',
    'IDENTIFIABLE INDIVIDUAL',
    'ONLINE IDENTIFIER',
    'PERSONAL DATA',
    'products obtained',
    'AUTOMATED PROCESSING',
    'Request for Pregnancy Disability Leave',
    'ANALYZE',
    'ECONOMIC SITUATION',
    'PERSONAL PREFERENCES',
    'religious beliefs',
    'RELIABILITY',
    'LOCATION',
    'MOVEMENTS',
    'PHYSICAL HEALTH CONDITION',
    'DIAGNOSIS',
    'credit card number',
    'CITIZENSHIP STATUS',
    'GENETIC',
    'last name',
    'mobile ad identifiers',
    'HEALTH-CARE',
    'PATIENT',
    'fingerprints',
    'products considered',
    'physical description',
    'voiceprint',
    'eye retinas',
    'Unique identifier',
    'consuming tendencies',
    'faceprint',
    'driver\'s license number',
    'Services considered',
    'global positioning system',
    'latitude',
    'longitude',
    'coordinates',
    'interests',
    'financial account',
    'user alias',
    'irises',
    'Request for leave for an employee\'s own serious health condition',
    'condition',
    'physical',
    'diagnosis',
    'insurance policy number',
    'immigration status',
    'known child',
    'sex life',
    'height',
    'AIDS',
    'medical diagnosis',
    'religious grooming practices',
    'Identifiable individual',
    'debit card',
    'ethnic',
    'origin',
    'medical treatment',
    'Inferences',
    'medical history',
    'mental health',
    'physical health',
    'mental',
    'Biometric information',
    'email content',
    'physical representation',
    'biological pattern',
    'mother\'s maiden name',
    'interaction with an internet website application',
    'deoxyribonucleic acid',
    'purchasing histories',
    'Disability',
    'targeting of advertising',
    'movements',
    'Hair color',
    'digital representation',
    'Initials',
    'SPECIFIC GEOLOCATION DATA',
    'driver authorization card number',
    'identification card number',
    'debit card number',
    'health insurance identification number',
    'user name',
    'Request for family care leave',
    'date of birth',
    'place of birth',
    'unique biometric',
    'human body',
    'HIV',
    'biometric',
    'language',
    'household',
    'driver\'s license number',
    'government-issued identification number',
    'driver license',
    'nondriver State identification card number',
    'individual taxpayer identification number',
    'military identification card number',
    'gait patterns',
    'unique personal identifier',
    'passwords',
    'personal identification number',
    'services obtained',
    'wellness program',
    'health promotion',
    'disease prevention',
    'health insurance policy number']


In [27]:
#make sure the lexixon terms are lower case 
for i in range(len(PII)):
    PII[i] = PII[i].lower()

In [38]:
df['negation_Extracted'] = df['negation_Extracted'].astype(str)

In [39]:
#filter out all of the PII terms and put them in a new column 

extracted =[] 
for index, row in df.iterrows():
    tempList =[] 
    rowSplit = row['negation_Extracted'].split(" ")
    for val in rowSplit:
        if val in PII:
            tempList.append(val)
    if len(tempList)>0:
        extracted.append(' '.join(tempList))
    else:
        extracted.append(np.nan)

df['PII_Extracted'] = extracted

In [40]:
#drop nans 

df = df[df['PII_Extracted'].notna()]

In [41]:
df['PII_Extracted']

0                                             behavior
4                         address name address address
6         address child sex name name medicine cookies
7    employment interests preferences condition fam...
8    employment interests preferences condition fam...
Name: PII_Extracted, dtype: object

In [42]:
#make sure to change the file name for the correct year 

df.PII_Extracted.to_csv('/content/drive/MyDrive/RESEARCH/CODE AND DATA/Pirvacy_Policies_OT/PoCS/data/filtered_pii_extracted/1997_pii_extracted.csv') 